In [1]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix


np.random.seed(42)

Using TensorFlow backend.


In [2]:
drive.mount('/content/Drive/',force_remount =  True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/MFCC Data/"

In [0]:
#Loading training pickle files
sound_lists_pkl_in = open(dirpath+"ca_data_padded_tr.pkl", "rb")
ca_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"en_data_padded_tr.pkl", "rb")
en_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"fr_data_padded_tr.pkl", "rb")
fr_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"it_data_padded_tr.pkl", "rb")
it_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"de_data_padded_tr.pkl", "rb")
de_data_tr = pickle.load(sound_lists_pkl_in)



In [0]:
#Loading test pickle files
sound_lists_pkl_in = open(dirpath+"ca_data_padded_te.pkl", "rb")
ca_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"en_data_padded_te.pkl", "rb")
en_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"fr_data_padded_te.pkl", "rb")
fr_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"it_data_padded_te.pkl", "rb")
it_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"de_data_padded_te.pkl", "rb")
de_data_te = pickle.load(sound_lists_pkl_in)

In [0]:
final_data_tr = np.concatenate((ca_data_tr, en_data_tr, fr_data_tr, it_data_tr, de_data_tr), axis = 0)

In [0]:
final_data_te = np.concatenate((ca_data_te, en_data_te, fr_data_te, it_data_te, de_data_te), axis = 0)

In [8]:
print(final_data_tr.shape)
print(final_data_te.shape)

(10000, 2)
(2500, 2)


In [0]:
del(de_data_te)
del(ca_data_te)
del(en_data_te)
del(fr_data_te)
del(it_data_te)

In [0]:
del(de_data_tr)
del(ca_data_tr)
del(en_data_tr)
del(fr_data_tr)
del(it_data_tr)

In [0]:
# final_data_tr

In [0]:
np.random.shuffle(final_data_tr)
np.random.shuffle(final_data_te)

In [0]:
# data_padded_pkl = open(dirpath+"final_data_tr.pkl", "wb")
# pickle.dump(final_data_tr, data_padded_pkl)

In [0]:
# data_padded_pkl = open(dirpath+"final_data_te.pkl", "wb")
# pickle.dump(final_data_te, data_padded_pkl)

### Labels

In [15]:

labels_tr =[]
for i in range(len(final_data_tr)):

      labels_tr.append(final_data_tr[i][1])  

labels_tr = np.array(labels_tr)
labels_tr

classnames, indices = np.unique(labels_tr, return_inverse=True)
print(indices)

final_labels_tr = indices

[4 2 0 ... 2 0 4]


In [16]:
labels_te =[]
for i in range(len(final_data_te)):

      labels_te.append(final_data_te[i][1])  

labels_te = np.array(labels_te)
labels_te

classnames, indices = np.unique(labels_te, return_inverse=True)
print(indices)

final_labels_te = indices

[3 2 1 ... 1 3 3]


In [17]:
final_X_tr =[]
for i in range(len(final_data_tr)):

      final_X_tr.append(final_data_tr[i][0])  

final_X_tr = np.array(final_X_tr)
final_X_tr[0].shape




(972, 40)

In [18]:
final_X_te =[]
for i in range(len(final_data_te)):

      final_X_te.append(final_data_te[i][0])  

final_X_te = np.array(final_X_te)
final_X_te[0].shape


(972, 40)

### Keras Model

In [0]:
from keras.layers.core import Dense, Permute, Reshape, Flatten 
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical

In [20]:
model = Sequential()
weight_decay = 0.001

model.add(Convolution2D(32, kernel_size=(4,4), strides=(1,1), activation="relu", padding="same", kernel_regularizer=l2(weight_decay), input_shape=(972,40,1)))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(64, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))



model.add(Convolution2D(128, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(256, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

model.add(Convolution2D(512, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))


model.add(Convolution2D(512, kernel_size=(4,4), strides=(1,1), activation="relu", kernel_regularizer=l2(weight_decay), padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

print(model.layers[-1].output_shape)

#model.add(Permute((2,1,3)))

print(model.layers[-1].output_shape)

bs, x, y, c = model.layers[-1].output_shape

model.add(Reshape([x,y*c]))

print(model.layers[-1].output_shape)

#model.add(LSTM(256, activation="tanh", return_sequences=True))

model.add(LSTM(512, activation="relu",dropout = 0.7, return_sequences=False))

# model.add(Flatten())

# model.add(Dense(256, activation="relu"))

# model.add(Dropout(rate=0.7))

model.add(Dense(5, activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.
(None, 16, 1, 512)
(None, 16, 1, 512)
(None, 16, 512)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 972, 40, 32)       544       
_________________________________________________________________
batch_normalization_1 (Batch (None, 972, 40, 32)       128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 486, 20, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 486, 20, 64)       32832     
_________________________________________________________________
batch_normalization_2 (Batch (None, 486, 20, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 243, 10, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 243, 10, 128)      131200    
__________

In [0]:
del(final_data_tr)
del(final_data_te)

In [0]:
tx = np.array(final_X_tr[:8000])
cvx = np.array(final_X_tr[8000:])
ty = np.array(final_labels_tr[:8000])
cvy = np.array(final_labels_tr[8000:])

In [0]:
del(final_labels_tr)
del(final_X_tr)

In [0]:
cvy = to_categorical(cvy)
ty = to_categorical(ty)

In [26]:
ty

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0002),
             metrics=["accuracy"])

In [28]:
#Fitting the model
model.fit(tx.reshape(-1,972,40,1), ty,
          batch_size=300,
          epochs=150,
          verbose=1,
          validation_data=(cvx.reshape(-1,972,40,1),cvy)
         )
          

Instructions for updating:
Use tf.cast instead.
Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 59s 7ms/step - loss: 2.5358 - acc: 0.4371 - val_loss: 2.0441 - val_acc: 0.6585
Epoch 2/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.9435 - acc: 0.6244 - val_loss: 1.8251 - val_acc: 0.6525
Epoch 3/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.7003 - acc: 0.7024 - val_loss: 1.6286 - val_acc: 0.7150
Epoch 4/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.5762 - acc: 0.7464 - val_loss: 1.6220 - val_acc: 0.7140
Epoch 5/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.4871 - acc: 0.7921 - val_loss: 1.5064 - val_acc: 0.7830
Epoch 6/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.4401 - acc: 0.8095 - val_loss: 1.7139 - val_acc: 0.7065
Epoch 7/150
8000/8000 [==============================] - 39s 5ms/step - loss: 1.3823 - a

In [0]:
y_pred = model.predict(cvx.reshape(-1,972,40,1))
matrix = confusion_matrix(cvy.argmax(axis=1), y_pred.argmax(axis=1))

In [30]:
matrix

array([[389,   9,   0,   2,   0],
       [ 22, 361,   0,   0,   0],
       [ 22,  13, 317,  44,   1],
       [ 19,  14,  26, 342,   0],
       [  5,   0,   0,   1, 413]])

In [31]:
np.unique(ty, return_counts=True)

(array([0., 1.], dtype=float32), array([32000,  8000]))

In [32]:
np.unique(cvy, return_counts=True)
  

(array([0., 1.], dtype=float32), array([8000, 2000]))